In [1]:
import mmcv
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pycocotools.coco import COCO
import numpy as np
import cv2
import albumentations as A

In [2]:
# config file 들고오기
config = 'upernet_swin_large_patch4_window12_512x512_80k_coco_pretrain_384x384_22K'
cfg = Config.fromfile(f'./configs/swin/{config}.py')

root = '../input/data/'

epoch = 'best_mIoU_iter_58000'

# dataset config 수정
# cfg.data.test.classes = classes
# cfg.data.test.img_prefix = root
# cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.data_root = root
cfg.data.test.img_dir = 'test/'
# cfg.data.val.ann_dir = 'test/'
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resized
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=1995
cfg.gpu_ids = [1]
cfg.work_dir = f'./work_dirs/{config}'

# cfg.model.roi_head.bbox_head.num_classes = 10
# cfg.model.bbox_head.num_classes = 10
# for head in cfg.model.roi_head.bbox_head:
#     head.num_classes = 10

# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2021-11-02 20:41:14,373 - mmseg - INFO - Loaded 819 images


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/segmentation/mmsegmentation/mmseg/models/backbones/swin.py:556: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '


Use load_from_local loader


In [19]:
output = single_gpu_test(model, data_loader) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 0.4 task/s, elapsed: 2262s, ETA:     0s

In [20]:
def test(idx, coco):
    size = 256
    transform = A.Compose([A.Resize(size, size)])

    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)

    image_id = coco.getImgIds(imgIds=idx)
    image_infos = coco.loadImgs(image_id)[0]
    file_name = image_infos['file_name']

    outs = output[idx] # mask
    img = cv2.imread('/opt/ml/segmentation/input/data/'+file_name) # image

    transformed = transform(image=img, mask=outs, interpolation=2)
    mask = transformed['mask'].reshape((256 * 256)).astype(int)

    file_name_list.append(file_name)
    preds_array = np.vstack((preds_array, mask))
    return file_name_list, preds_array

In [21]:
# sample_submisson.csv 열기
submission = pd.read_csv('/opt/ml/segmentation/baseline_code/submission/sample_submission.csv', index_col=None)
coco = COCO('/opt/ml/segmentation/input/data/test.json')

# test set에 대한 prediction
for idx in range(len(output)):
    file_names, preds = test(idx, coco)

    # PredictionString 대입
    for file_name, string in zip(file_names, preds):
        submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())},
                                        ignore_index=True)

# submission.csv로 저장
submission.to_csv(cfg.work_dir + "/submission/submission_42k.csv", index=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/opt/conda/envs/segmentation/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
